#<font color='orange'>Симнете го податочното множество</font>

In [ ]:
!gdown 1-K5XbNHNLYBQG9VyTQUzxov_x5AmoITF

Downloading...
From: https://drive.google.com/uc?id=1-K5XbNHNLYBQG9VyTQUzxov_x5AmoITF
To: /content/IMDB_5.csv
100% 323k/323k [00:00<00:00, 128MB/s]


#<font color='orange'>Направете претпроцесирање на податоците</font>

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("./IMDB_5.csv")
df.head()

,review_es,sentiment
0,"¡Esta película es horrible, ni siquiera puedo ...",negative
1,Why do movie makers always go against the auth...,negative
2,No puedo creer que aquellos alabando esta pelí...,negative
3,Esta película realmente usó sus ubicaciones bi...,positive
4,"De manera extraña, esta película nunca llegó a...",positive


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['sentiment'] = encoder.fit_transform(df['sentiment'])

In [ ]:
df.head()

,review_es,sentiment
0,"¡Esta película es horrible, ni siquiera puedo ...",0
1,Why do movie makers always go against the auth...,0
2,No puedo creer que aquellos alabando esta pelí...,0
3,Esta película realmente usó sus ubicaciones bi...,1
4,"De manera extraña, esta película nunca llegó a...",1


In [ ]:
df['sentiment'].value_counts()

1    132
0    118
Name: sentiment, dtype: int64

#<font color='orange'>Поделето го на тренирачко и тестирачко множество во сооднос 80:20</font>

In [ ]:
!pip install transformers datasets 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df.columns = ['review_es', 'label']

In [ ]:
from sklearn.model_selection import train_test_split

df_train,df_test = train_test_split(df, test_size=0.2)

In [ ]:
df_train.to_csv('train.csv',index=False)
df_test.to_csv('test.csv',index=False)

In [ ]:
dataset = load_dataset("csv", data_files={"train": "train.csv", "test": "test.csv"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-015be06cc23b0a88/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review_es', 'label'],
        num_rows: 200
    })
    test: Dataset({
        features: ['review_es', 'label'],
        num_rows: 50
    })
})

#<font color='orange'>Искористете соодветен трансформер модел за проблемот</font>

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')
encoded_dataset = dataset.map(lambda t: tokenizer(t['review_es'],  truncation=True), batched=True,load_from_cache_file=False)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('dccuchile/bert-base-spanish-wwm-cased',num_labels =2)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

#<font color='orange'>Фино-подесете (тренирајте) го моделот</font>

In [ ]:
from transformers import TrainingArguments, Trainer
arg = TrainingArguments(
    "sentiment",
    num_train_epochs=5,
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,
    seed=42
)
trainer = Trainer(
    model=model,
    args=arg,
    tokenizer=tokenizer,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test']
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: review_es. If review_es are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 200
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 125
  Number of trainable parameters = 109852418
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=125, training_loss=0.2622578125, metrics={'train_runtime': 88.1659, 'train_samples_per_second': 11.342, 'train_steps_per_second': 1.418, 'total_flos': 246066392304960.0, 'train_loss': 0.2622578125, 'epoch': 5.0})

#<font color='orange'>Тестирајте го моделот на тестирачкото множество</font>

In [ ]:
predictions = trainer.predict(encoded_dataset["test"])
preds = np.argmax(predictions.predictions, axis=-1)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: review_es. If review_es are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 50
  Batch size = 8


#<font color='orange'>Пресметајте f1 score</font>

In [ ]:
from sklearn.metrics import classification_report, f1_score
print(classification_report(predictions.label_ids, preds))
print(f'f1 score {f1_score(predictions.label_ids, preds)}')

              precision    recall  f1-score   support

           0       0.89      0.76      0.82        21
           1       0.84      0.93      0.89        29

    accuracy                           0.86        50
   macro avg       0.87      0.85      0.85        50
weighted avg       0.86      0.86      0.86        50

f1 score 0.8852459016393444


Искористив шпански берт. Зголемувањето на бројот на епохи значително помогна со перформанците. Тоа значи дека моделот правел underfit. Со околу 5 епохи најдов најдобри резултати со f1 od 0.88

Дополнително го повторив истово и со multilingual модел на берт

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-uncased')
encoded_dataset = dataset.map(lambda t: tokenizer(t['review_es'],  truncation=True), batched=True,load_from_cache_file=False)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-uncased/snapshots/800c34f3d5aa174fe531f560b44b8d14592225b7/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_si

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-multilingual-uncased',num_labels =2)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-uncased/snapshots/800c34f3d5aa174fe531f560b44b8d14592225b7/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_si

In [ ]:
from transformers import TrainingArguments, Trainer
arg = TrainingArguments(
    "sentiment",
    num_train_epochs=5,
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,
    seed=42
)
trainer = Trainer(
    model=model,
    args=arg,
    tokenizer=tokenizer,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test']
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: review_es. If review_es are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 200
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch s

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=125, training_loss=0.5182033081054688, metrics={'train_runtime': 89.6589, 'train_samples_per_second': 11.153, 'train_steps_per_second': 1.394, 'total_flos': 242880281868960.0, 'train_loss': 0.5182033081054688, 'epoch': 5.0})

In [ ]:
predictions = trainer.predict(encoded_dataset["test"])
preds = np.argmax(predictions.predictions, axis=-1)
print(classification_report(predictions.label_ids, preds))
print(f'f1 score {f1_score(predictions.label_ids, preds)}')

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: review_es. If review_es are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 50
  Batch size = 8


              precision    recall  f1-score   support

           0       0.70      0.76      0.73        21
           1       0.81      0.76      0.79        29

    accuracy                           0.76        50
   macro avg       0.76      0.76      0.76        50
weighted avg       0.76      0.76      0.76        50

f1 score 0.7857142857142857


шпанскиот модел погоре успева поголем f1score oд 0.885 во споредба со овој multilingual од 0.785. Иако имаше некои текстови кои се на англиски шпанскиот модел е подобар за овој проблем.